In [1]:
from datetime import datetime,timezone,timedelta
import sys
import tweepy
import re
import os
import json

In [ ]:
# Import the necessary methods from tweepy library
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import jsonpickle
import twitter

#!!!USER SPECIFIC
CONSUMER_KEY = ''
CONSUMER_SECRET = '' 
ACCESS_TOKEN = '' 
ACCESS_TOKEN_SECRET = ''  

#output folder-user-specific
output_folder = 'Seminar/'

##########

#Pass our consumer key and consumer secret to Tweepy's user authentication handler
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)

#Pass our access token and access secret to Tweepy's user authentication handler
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

# Prep for getting tweets from the US
api = tweepy.API(auth)
places = api.geo_search(query="USA", granularity="country")
place_id = places[0].id
print('USA id is: ',place_id)

#Switching to application authentication to get 450 tweets every 15 min (user-authentication allows 180)
auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Problem Connecting to API") #error handling
api.rate_limit_status()['resources']['search'] # Check how many queries you have left 

# Define what we search for (hashtags and phrases)
#searchQuery = 'place:96683cc9126741d1 #Pepsi OR #Coca-Cola OR' \
              #'"Pepsi" OR "Coca-Cola"'
searchQuery = 'place:96683cc9126741d1 #Bitcoin OR #BTC OR' \
              '"Bitcoin" OR "BTC"'
maxTweets = 5000 # Max number of tweets to collect 
tweetsPerQry = 100  # The Search API allows up to 100 tweets per query 

tweetCount = 0

# Open a text file to save the tweets to in JSON format
with open(output_folder + 'tweets_Histric_Data.json', 'w') as f:

    for tweet in tweepy.Cursor(api.search,q=searchQuery).items(maxTweets) :         

        #Verify the tweet has place info before writing (It should, if it got past our place filter)
        if tweet.place is not None:
            if tweet.created_at >= beginday_datetime_utc and tweet.created_at <= endday_datetime_utc :
            
            #Write the JSON format to the text file, and add one to the number of tweets we've collected
                f.write(jsonpickle.encode(tweet._json, unpicklable=False) + '\n')
                tweetCount += 1

    #Display how many tweets we have collected
    print("Downloaded {0} tweets".format(tweetCount))

# have a look at the data        
tweet._json

#A function to clean up and print all of the JSON attributes 
def PrintMembers(obj):
    for attribute in dir(obj):
        #We don't want to show built in methods of the class
        if not attribute.startswith('__'):
            print(attribute)
            
PrintMembers(tweet)
PrintMembers(tweet.place)

In [ ]:
import pandas as pd
df = pd.read_json(r'~/Seminar/tweets_Histric_Data.json')
import json
data = []
with open('/Users/candice/Desktop/Seminar/tweets_Histric_Data.json', errors='ignore') as f:
    for line in f:
        data.append(json.loads(line))
data

In [ ]:
pd.DataFrame(data)[['created_at','text']]

In [ ]:
def analyze_sentiment_vader_lexicon(post, 
                                    threshold = 0.1,
                                    verbose = False):
    
    #pre-process text
    post = normalize_accented_characters(post)
    post = html_parser.unescape(post)
    post = strip_html(post)
    
    #analyze the sentiment for posts
    analyzer = SentimentIntensityAnalyzer()
    scores = analyzer.polarity_scores(review)
    
    #get binary sentiment
    binary_sentiment = 'positive' if scores['compound'] >= threshold\
                                   else 'negative'
    
    if verbose:
        
        #display sentiment 
        sentiment_frame = pd.DataFrame([[binary_sentiment, round(scores['compound'], 2)]],
                                        columns=pd.MultiIndex(levels=[['SENTIMENT STATS:'], 
                                                                      ['Binary Sentiment ', 'Polarity Score']], 
                                                              labels=[[0,0],[0,1]]))
        print(sentiment_frame.to_string(index=False))
    
    return binary_sentiment,scores['compound']  

In [10]:

beginday = "2018-01-01"
endday = "2018-01-31"
beginday_datetime_string = beginday + " 00:00:00" 
endday_datetime_string = endday + " 23:59:59"  
outputFile_name0 = beginday + "-" + endday  
outputFile_name1 = ".js"  
outputFile_name = outputFile_name0 + outputFile_name1

beginday_datetime = datetime.strptime(beginday_datetime_string,'%Y-%m-%d %H:%M:%S')
endday_datetime = datetime.strptime(endday_datetime_string,'%Y-%m-%d %H:%M:%S')


beginday_datetime_beijing = beginday_datetime.replace(tzinfo=timezone(timedelta(hours=8)))
endday_datetime_beijing = endday_datetime.replace(tzinfo=timezone(timedelta(hours=8))) 
beginday_datetime_utc = beginday_datetime_beijing.astimezone(timezone.utc) 
endday_datetime_utc = endday_datetime_beijing.astimezone(timezone.utc) 
beginday_datetime_utc = beginday_datetime_utc.replace(tzinfo=None) 
endday_datetime_utc = endday_datetime_utc.replace(tzinfo=None) 

consumer_key = ''
consumer_secret = '' 
access_token = '' 
access_secret = ''  

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

searchQuery = 'place:96683cc9126741d1 #Bitcoin OR #BTC OR' \
              '"Bitcoin" OR "BTC"'
tweets = []

for tweet in tweepy.Cursor(api.user_timeline,q=searchQuery,tweet_mode="extended").items():
    if tweet.created_at >= beginday_datetime_utc and tweet.created_at <= endday_datetime_utc :
        tweets.append(tweet._json)

with open(outputFile_name,'w') as outputFile:
        print("Grailbird.data.tweets_{0} =".format(outputFile_name0),file = outputFile)
        tweets_output = json.dumps(tweets, sort_keys=True, indent=4, separators=(',', ':'))
        tweets_output = re.sub('"full_text":"','"text":"',tweets_output) 
        print(tweets_output,file = outputFile)   
